In [21]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec, FastText
###import glove
###from glove import Corpus

import collections
import gc 

import warnings
warnings.filterwarnings('ignore')

In [2]:
#pip install -U gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 31.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
#pip install glove-py

  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for glove-py: filename=glove_py-0.2.3-cp37-cp37m-macosx_10_9_x86_64.whl size=78657 sha256=655a0cc0bb130b0decd1d920672a7e1b0e6b21fdf8d84d8c8d92e37e142fc788
  Stored in directory: /Users/aneezzakaria/Library/Caches/pip/wheels/f4/03/3d/bb3b646c1fadfecf585bcf02be3920ac8322e958bf2637cfcf
Successfully built glove-py
Note: you may need to restart the kernel to use updated packages.


In [22]:
new_notes = pd.read_pickle("data/ner_df.p") # med7
w2vec = Word2Vec.load("embeddings/word2vec.model")
fasttext = FastText.load("embeddings/fasttext.model")

In [23]:
null_index_list = []
for i in new_notes.itertuples():
    
    if len(i.ner) == 0:
        null_index_list.append(i.Index)
new_notes.drop(null_index_list, inplace=True)

In [24]:
med7_ner_data = {}

for ii in new_notes.itertuples():
    
    p_id = ii.SUBJECT_ID
    ind = ii.Index
    
    try:
        new_ner = new_notes.loc[ind].ner
    except:
        new_ner = []
            
    unique = set()
    new_temp = []
    
    for j in new_ner:
        for k in j:
            
            unique.add(k[0])
            new_temp.append(k)

    if p_id in med7_ner_data:
        for i in new_temp:
            med7_ner_data[p_id].append(i)
    else:
        med7_ner_data[p_id] = new_temp

In [25]:
pd.to_pickle(med7_ner_data, "data/new_ner_word_dict.pkl")

In [26]:
def mean(a):
    return sum(a) / len(a)

In [28]:
data_types = [med7_ner_data]
data_names = ["new_ner"]

for data, names in zip(data_types, data_names):
    new_word2vec = {}
    print("w2vec starting..")
    for k,v in data.items():

        patient_temp = []
        for i in v:
            try:
                patient_temp.append(w2vec.wv[i[0]])
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec.wv[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: continue
                    avg = np.asarray(avg)
                    t = np.asarray(list(map(mean, zip(*avg))))
                    patient_temp.append(t)
        if len(patient_temp) == 0: continue
        new_word2vec[k] = patient_temp

    #############################################################################
    print("fasttext starting..")
        
    new_fasttextvec = {}

    for k,v in data.items():

        patient_temp = []

        for i in v:
            try:
                patient_temp.append(fasttext.wv[i[0]])
            except:
                pass
        if len(patient_temp) == 0: continue
        new_fasttextvec[k] = patient_temp

    #############################################################################    
        
    print("combined starting..")
    new_concatvec = {}

    for k,v in data.items():
        patient_temp = []
        #if k != 6: continue
        for i in v:
            w2vec_temp = []
            fasttemp = []
            try:
                w2vec_temp = w2vec.wv[i[0]]
            except:
                avg = []
                num = 0
                temp = []

                if len(i[0].split(" ")) > 1:
                    for each_word in i[0].split(" "):
                        try:
                            temp = w2vec.wv[each_word]
                            avg.append(temp)
                            num += 1
                        except:
                            pass
                    if num == 0: 
                        w2vec_temp = [0] * 100
                    else:
                        avg = np.asarray(avg)
                        w2vec_temp = np.asarray(list(map(mean, zip(*avg))))
                else:
                    w2vec_temp = [0] * 100

            try:
                fasttemp = fasttext.wv[i[0]]
            except:
                fasttemp = [0] * 100
            ##print(fasttemp)

            appended = np.append(fasttemp, w2vec_temp, 0)
            patient_temp.append(appended)
        if len(patient_temp) == 0: continue
        new_concatvec[k] = patient_temp

    print(len(new_word2vec), len(new_fasttextvec), len(new_concatvec))
    pd.to_pickle(new_word2vec, "data/"+names+"_word2vec_dict.pkl")
    pd.to_pickle(new_fasttextvec, "data/"+names+"_fasttext_dict.pkl")
    pd.to_pickle(new_concatvec, "data/"+names+"_combined_dict.pkl")

w2vec starting..
fasttext starting..
combined starting..
21731 21657 21837


In [29]:
diff = set(new_fasttextvec.keys()).difference(set(new_word2vec))
for i in diff:
    del new_fasttext[i]
    del new_combined[i]
print (len(new_word2vec), len(new_fasttextvec), len(new_concatvec))


pd.to_pickle(new_word2vec, "data/"+"new_ner"+"_word2vec_limited_dict.pkl")
pd.to_pickle(new_fasttextvec, "data/"+"new_ner"+"_fasttext_limited_dict.pkl")
pd.to_pickle(new_concatvec, "data/"+"new_ner"+"_combined_limited_dict.pkl")

21731 21657 21837
